<a href="https://colab.research.google.com/github/LalitoW/seguridad-final/blob/main/proyecto_final_hashing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import time

class Block:
    def __init__(self, index, previous_hash, timestamp, data, difficulty):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.difficulty = difficulty
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        # Combine all block data into a single string
        block_string = (str(self.index) +
                        self.previous_hash +
                        str(self.timestamp) +
                        self.data +
                        str(self.nonce))

        # Apply SHA-256 hash
        return hashlib.sha256(block_string.encode()).hexdigest()

    def mine_block(self):
        # The Target: A hash starting with 'difficulty' number of zeros
        target = "0" * self.difficulty

        print(f"Mining Block {self.index} with difficulty {self.difficulty}...")
        start_time = time.time()

        # Brute force: Keep changing nonce until the hash matches the target
        while self.hash[:self.difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()

        end_time = time.time()
        print(f"Block Mined! Nonce: {self.nonce}")
        print(f"Hash: {self.hash}")
        print(f"Time taken: {end_time - start_time:.4f} seconds\n")

# --- simulation ---

# 1. Create Genesis Block
difficulty_level = 6  # Increase this to 5 or 6 to see drastic time increases
genesis_block = Block(0, "0", time.time(), "Genesis Block", difficulty_level)
genesis_block.mine_block()

# 2. Create Second Block (Linked to Genesis)
block_2 = Block(1, genesis_block.hash, time.time(), "Transaction: Alice pays Bob 5 BTC", difficulty_level)
block_2.mine_block()

# 3. Create Third Block (Linked to Second)
block_3 = Block(2, block_2.hash, time.time(), "Transaction: Bob pays Charlie 2 BTC", difficulty_level)
block_3.mine_block()

# 4. Demonstrate Integrity Check
print("-" * 30)
print("INTEGRITY CHECK:")
print(f"Block 2 stored Previous Hash: {block_2.previous_hash}")
print(f"Actual Genesis Hash:          {genesis_block.hash}")

if block_2.previous_hash == genesis_block.hash:
    print("Result: Chain is Valid.")
else:
    print("Result: Chain is Broken!")

# 5. Demonstrate Avalanche Effect (Tampering)
print("-" * 30)
print("TAMPERING ATTEMPT:")
print("Changing data in Genesis Block...")
genesis_block.data = "Genesis Block (HACKED)"
# The hash must be recalculated because data changed
new_genesis_hash = genesis_block.calculate_hash()

print(f"New Genesis Hash:             {new_genesis_hash}")
print(f"Block 2 stored Previous Hash: {block_2.previous_hash}")

if block_2.previous_hash == new_genesis_hash:
    print("Result: Chain is still Valid.")
else:
    print("Result: Chain BROKEN. Tampering detected!")

Mining Block 0 with difficulty 6...
Block Mined! Nonce: 41555157
Hash: 00000047d1af4318f770ca8ffde359d510478f970679648a59e6c85b0c17ce8e
Time taken: 132.8848 seconds

Mining Block 1 with difficulty 6...
Block Mined! Nonce: 2549532
Hash: 0000004ca2e9084bed36fb06b19e466b48c00d6c7b7bb5558b46cd70ba7e19ef
Time taken: 10.5964 seconds

Mining Block 2 with difficulty 6...
Block Mined! Nonce: 3277313
Hash: 00000077f457515ea7e8593b691712b09b9910482a903c403db85b5cc55d880d
Time taken: 12.9573 seconds

------------------------------
INTEGRITY CHECK:
Block 2 stored Previous Hash: 00000047d1af4318f770ca8ffde359d510478f970679648a59e6c85b0c17ce8e
Actual Genesis Hash:          00000047d1af4318f770ca8ffde359d510478f970679648a59e6c85b0c17ce8e
Result: Chain is Valid.
------------------------------
TAMPERING ATTEMPT:
Changing data in Genesis Block...
New Genesis Hash:             d2a30de45acd0a5d0fba3b276a4d5cb56e0c7143d49b0274b0dc795b042b305f
Block 2 stored Previous Hash: 00000047d1af4318f770ca8ffde359d5104

In [1]:
import hashlib
import binascii

# These two different hex strings are a known MD5 collision pair
# Note the slight differences in the byte sequences (e.g., '200' vs '202')
hex_data_1 = '4dc968ff0ee35c209572d4777b721587d36fa7b21bdc56b74a3dc0783e7b9518afbfa200a8284bf36e8e4b55b35f427593d849676da0d1555d8360fb5f07fea2'
hex_data_2 = '4dc968ff0ee35c209572d4777b721587d36fa7b21bdc56b74a3dc0783e7b9518afbfa202a8284bf36e8e4b55b35f427593d849676da0d1d55d8360fb5f07fea2'

# Convert hex to binary data
data1 = binascii.unhexlify(hex_data_1)
data2 = binascii.unhexlify(hex_data_2)

print("--- MD5 COLLISION CHECK ---")
print(f"MD5(Data 1): {hashlib.md5(data1).hexdigest()}")
print(f"MD5(Data 2): {hashlib.md5(data2).hexdigest()}")

print("\n--- SHA-256 SECURITY CHECK ---")
print(f"SHA-256(Data 1): {hashlib.sha256(data1).hexdigest()}")
print(f"SHA-256(Data 2): {hashlib.sha256(data2).hexdigest()}")

if hashlib.md5(data1).hexdigest() == hashlib.md5(data2).hexdigest():
    print("\n[!] CRITICAL FAILURE: MD5 Collision Detected!")
else:
    print("\n[ok] MD5 is secure.")

if hashlib.sha256(data1).hexdigest() != hashlib.sha256(data2).hexdigest():
    print("[ok] SHA-256 correctly distinguishes the data.")

--- MD5 COLLISION CHECK ---
MD5(Data 1): 008ee33a9d58b51cfeb425b0959121c9
MD5(Data 2): 008ee33a9d58b51cfeb425b0959121c9

--- SHA-256 SECURITY CHECK ---
SHA-256(Data 1): 54bcb9a4fda31e4f254303e3959acd5e420ad18a80949d56a3000c3716fbd1a0
SHA-256(Data 2): 90774a6455a2bdb7d106e533923ecbefe81392ca55bed0ce81cfab2c1a7f0afe

[!] CRITICAL FAILURE: MD5 Collision Detected!
[ok] SHA-256 correctly distinguishes the data.
